In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Loading the Dataset

In [ ]:
df = pd.read_csv('/kaggle/input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv')
df

## Checking the Info of Dataset

In [ ]:
df.info()

In [ ]:
lc = []
ln = []
for i in df.columns:
    if df[i].dtype == 'object':
        lc.append(i)
    else:
        ln.append(i)
        
print('The Categorical Values are',lc)
print('The no.of Categorical Values are',len(lc))
print('The Numerical Values are',ln)
print('The no.of Numerical Values are',len(ln))

## Finding Null Values in the Dataset

In [ ]:
df.isnull().sum()

There are 67 Null values present in 'salary' column

In [ ]:
print(df['gender'].value_counts())
print(df['ssc_b'].value_counts())
print(df['hsc_b'].value_counts())
print(df['hsc_s'].value_counts())
print(df['degree_t'].value_counts())
print(df['workex'].value_counts())
print(df['specialisation'].value_counts())
print(df['status'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.countplot(df['status'],hue=df['gender'])

From the countplot, we can see that male rate is higher in both placed list and Not placed list

In [ ]:
sns.boxplot(df['ssc_b'],df['salary'])

From the above box plot, we can see that candidates who studied ssc in 'Central' board are placed with high salaries

## Finding Outliers

In [ ]:
for i in df.columns:
    if df[i].dtype != 'object':
        sns.boxplot(df[i])
        plt.show()

Outliers are present in 'hsc_p','degree_p'

## Treating Outliers

In [ ]:
q3_hsc = df['hsc_p'].quantile(0.75)
q1_hsc = df['hsc_p'].quantile(0.25)

In [ ]:
IQR_hsc = q3_hsc - q1_hsc

In [ ]:
ul_hsc = q3_hsc+(1.5*IQR_hsc)
ll_hsc = q1_hsc-(1.5*IQR_hsc)

In [ ]:
df = df[(df['hsc_p']>ll_hsc)&(df['hsc_p']<ul_hsc)]

In [ ]:
q3_deg = df['degree_p'].quantile(0.75)
q1_deg = df['degree_p'].quantile(0.25)

In [ ]:
IQR_deg = q3_deg - q1_deg

In [ ]:
ul_deg = q3_deg+(1.5*IQR_deg)
ll_deg = q1_deg-(1.5*IQR_deg)

In [ ]:
df = df[(df['degree_p']>ll_deg)&(df['degree_p']<ul_deg)]

In [ ]:
df.shape

Outliers are removed in columns 'hsc_p' and 'degree_p'

## Dropping the unwanted rows and colummns in Dataset

In [ ]:
df1 = df.drop(columns = ['salary','sl_no','gender'],axis=1)
df1.shape

In [ ]:
df1.head()

Dropped 'sl_no','gender','salary' rows from the dataset

# Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
df1['ssc_b'] = le.fit_transform(df1['ssc_b'])
df1['hsc_b'] = le.fit_transform(df1['hsc_b'])
df1['hsc_s'] = le.fit_transform(df1['hsc_s'])
df1['degree_t'] = le.fit_transform(df1['degree_t'])
df1['workex'] = le.fit_transform(df1['workex'])
df1['specialisation'] = le.fit_transform(df1['specialisation'])
df1['status'] = le.fit_transform(df1['status'])

In [ ]:
df1

# seperating the Dataset into X and y

In [ ]:
X = df1.drop(['status'],axis=1)
y = df1['status']

Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

## Logistic Regresssion****

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(solver='lbfgs')

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score, classification_report

In [ ]:
auc = roc_auc_score(y_test,y_pred)

In [ ]:
print('Accuracy of Logistic Regression:',auc)

In [ ]:
print('Classification Report:\n',classification_report(y_test,y_pred))

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier()

In [ ]:
rfc.fit(X_train,y_train)

In [ ]:
y_pred_rfc = rfc.predict(X_test)

In [ ]:
auc_rfc = roc_auc_score(y_test,y_pred_rfc)

In [ ]:
print('The Accuracy for Random Forest Classifier:',auc_rfc)

In [ ]:
print('Classification Report:\n',classification_report(y_test,y_pred_rfc))

In [ ]:
print('The Accuracy score of Logistic Regression:',auc)
print('The Accuracy score of Random Forest Classifier:',auc_rfc)

From the above scores, Logistic Regression is having Good score (0.8264864864864865) when compared to score of Random Forest Classifier (0.7664864864864865).

We can go with Logistic Regression and Deploy the model